<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/pobieranie_zdj%C4%99%C4%87/pobieranie_zdj%C4%99%C4%87_z_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Przed uruchomieniem trzeba utworzyć folder "ImgData" w MyDrive



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!git clone https://github.com/PiotrMaciejKowalski/BigData2022-actors.git
!mv /content/BigData2022-actors/* .
!mv /content/BigData2022-actors/.* .
!rmdir /content/BigData2022-actors/

Cloning into 'BigData2022-actors'...
remote: Enumerating objects: 1665, done.
remote: Counting objects: 100% (443/443), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 1665 (delta 326), reused 306 (delta 224), pack-reused 1222
Receiving objects: 100% (1665/1665), 6.04 MiB | 17.87 MiB/s, done.
Resolving deltas: 100% (1014/1014), done.
mv: cannot move '/content/BigData2022-actors/.' to './.': Device or resource busy
mv: cannot move '/content/BigData2022-actors/..' to './..': Device or resource busy


In [3]:
!chmod +x setup_sparka.sh
!./setup_sparka.sh

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 18.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=43e1430d2c787bd02c0b304bb7f49169f82d9b681401527d58af067c5908b8c5
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2023-01-19 16:04:37--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 

In [4]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
import requests
import numpy as np
import cv2
from lib.pyspark_init import create_spark_context

In [6]:
spark = create_spark_context()

In [9]:
#wczytuje obrazy i zapisuje
def save_download(url, file_name):
    file_name=file_name+'.jpg'
    response = requests.get(url)
    with open(file_name, "wb") as f:
      f.write(response.content)
    cv2.imwrite('/content/gdrive/MyDrive/bigdataaktorzy/ImgData/'+file_name, cv2.imread('/content/'+file_name))

In [10]:
def df_save_download(dataframe, num_col, num_namecol):
  for i in range(dataframe.count()):
    save_download(dataframe.collect()[i][num_col], str(dataframe.collect()[i][num_namecol]))

test:

In [11]:
dataframe = [("Lucas","https://images.unsplash.com/photo-1508921912186-1d1a45ebb3c1?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8cGhvdG98ZW58MHx8MHx8&w=1000&q=80","Smith","36636","M",3000),
    ("Elijah","https://www.paperlessmovement.com/wp-content/uploads/2019/09/o2dvsv2pnhe.jpg","","40288","M",4000),
    ("Emma","https://mymodernmet.com/wp/wp-content/uploads/2019/12/photos-of-indonesia-rarindra-prakarsa-4.jpg","Williams","42114","F",4000),
    ("Sophia","https://live-production.wcms.abc-cdn.net.au/50749be1153e1907d7e1208fc96432f8?impolicy=wcms_crop_resize&cropH=844&cropW=1500&xPos=0&yPos=0&width=862&height=485","Jones","39192","F",4000),
    ("Mia","https://media.wired.com/photos/62ccbecc4847c5414f1e3dc9/3:2/w_1280%2Cc_limit/Light-Photo-Video-Like-a-Pro-Gear-GettyImages-142009824.jpg","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("name",StringType(),True), \
    StructField("url",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=dataframe,schema=schema)

In [12]:
df_save_download(df, 1, 0)